In [ ]:
import pandas as pd
from datetime import datetime
import re

In [ ]:
!wget -O data.csv "https://drive.google.com/uc?id=1QMCW2oZtr8dSmremdED6isc7SpvMsYmk"

--2023-11-22 04:38:37--  https://drive.google.com/uc?id=1QMCW2oZtr8dSmremdED6isc7SpvMsYmk
Resolving drive.google.com (drive.google.com)... 209.85.146.101, 209.85.146.139, 209.85.146.102, ...
Connecting to drive.google.com (drive.google.com)|209.85.146.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t278fl8rm6o3rv2dm4aaff8v1l1j4v1p/1700627850000/01439699083371114626/*/1QMCW2oZtr8dSmremdED6isc7SpvMsYmk?uuid=88feaf63-556f-4b20-80e0-e690f9f8dfb9 [following]
--2023-11-22 04:38:38--  https://doc-0s-08-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/t278fl8rm6o3rv2dm4aaff8v1l1j4v1p/1700627850000/01439699083371114626/*/1QMCW2oZtr8dSmremdED6isc7SpvMsYmk?uuid=88feaf63-556f-4b20-80e0-e690f9f8dfb9
Resolving doc-0s-08-docs.googleusercontent.com (doc-0s-08-docs.googleusercontent.com)... 173.194.197.132, 2607:f8b0:4001:c1b::84
Connecting to doc-0

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
len(df)

2231

In [ ]:
df.sample(10).T

,2042,2133,284,707,1947,369,298,737,1065,1855
Course-type,PhD / Doctorate,Short course,Bachelor's degree,Master's degree,Master's degree,Master's degree,Master's degree,Master's degree,Master's degree,Master's degree
Title,Saarbrücken Graduate School of Computer Science,Sustainable Development Academy | SDA,BSc Autonomy Technologies,Forest and Ecosystem Sciences (FES),Physics (MSc),Computational Linguistics,Building Sustainability in an Urban Future (M...,Geomatics,International Management and Intercultural Co...,MSc Physics
University,Saarland University,University of Potsdam,FAU Erlangen-Nürnberg,University of Göttingen,Technische Universität Dresden,University of Stuttgart,Technische Universität Berlin,Neubrandenburg University of Applied Sciences,TH Köln (University of Applied Sciences),FAU Erlangen-Nürnberg
Location,Saarbrücken,Potsdam,Erlangen,Göttingen,Dresden,Stuttgart,Berlin,Neubrandenburg,Köln,Erlangen
C-badge,NaN,NaN,NaN,NaN,NaN,Partly online,NaN,NaN,NaN,NaN
Subject,Computer Science,"Law, Economics and Social Sciences in general",Electrical Engineering,"Forestry, Wood Economics","Physics, Astronomy",Computer Science,Business and Economics,Surveying,Business and Economics,"Physics, Astronomy"
Costs,NaN,EUR 850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Date(s),NaN,"August, 2024 (Registration Deadline 14 June, 2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Language,English,English,English,English,English,English,English,English,English,English
Language level,NaN,"English: B2, C1, C2",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['Scholarships'] = df['C-badge'].apply(lambda x: 'Yes' if isinstance(x, str) and 'Scholarships' in x else 'No')

In [ ]:
df['Fully online'] = df['C-badge'].apply(lambda x: 'Yes' if isinstance(x, str) and 'Fully online' in x else 'No')

In [ ]:
df['Hybrid'] = df['C-badge'].apply(lambda x: 'Yes' if isinstance(x, str) and 'Hybrid' in x else 'No')

In [ ]:
df['Partly online'] = df['C-badge'].apply(lambda x: 'Yes' if isinstance(x, str) and 'Partly online' in x else 'No')

In [ ]:
df['Fully on-site'] = df['C-badge'].apply(lambda x: 'No' if isinstance(x, str) and ('Partly online' or 'Hybrid' or  'Fully online')  in x else 'Yes')

In [ ]:
def check_presence(x):
    strings_to_check = ['online', 'Hybrid']
    if isinstance(x, str):
        return 'No' if any(string in x for string in strings_to_check) else 'Yes'
    else:
        return 'Yes'

df['Fully on-site'] = df['C-badge'].apply(check_presence)


In [ ]:
df = df.rename(columns={'Location': 'City', 'University': 'Institution'})

In [ ]:
def costs_to_numbers(costs_str):
    try:
        costs_value = float(costs_str.split()[1].replace(',', ''))
        costs_numbers = costs_value * 121.03
        return costs_numbers
    except:
        return 0

df['Costs (Taka)'] = df['Costs'].apply(lambda x: costs_to_numbers(x))

In [ ]:
def tuition_to_numbers(tuition_str):
    try:
        tuition_value = float(tuition_str.split()[1].replace(',', ''))
        tuition_numbers = tuition_value * 121.03
        return tuition_numbers
    except:
        return 0

df['Tuition fees (Taka per semester)'] = df['Tuition fees per semester'].apply(lambda x: tuition_to_numbers(x))

In [ ]:
def semesters_to_numbers(semesters_str):
    try:
        num_semesters = int(semesters_str.split()[0])
        return num_semesters
    except:
        return None

df['Duration (Semesters)'] = df['Duration'].apply(lambda x: semesters_to_numbers(x))

In [ ]:
def semesters_to_months(semesters_str, months_per_semester=6):
    try:
        num_semesters = int(semesters_str.split()[0])
        equivalent_months = num_semesters * months_per_semester
        return equivalent_months
    except:
        return None

df['Duration (Months)'] = df['Duration'].apply(lambda x: semesters_to_months(x))


In [ ]:
df['Duration (Years)'] = df['Duration (Months)'].apply(lambda x: x/12)


In [ ]:
current_year = datetime.now().year

df['Course enroll End Date'] = df['Date(s)'].str.extract(r'(\s\d+\s\w+,\s\d+)')


df['Course enroll Start Date'] = df['Date(s)'].str.extract(r'(\d+\s\w+)') + df['Course enroll End Date'].str.extract(r'(,\s\d+)')


df['Registration Deadline'] = df['Date(s)'].str.extract(r'(Registration Deadline\s\d+\s\w+,\s\d+)')

df['Registration Deadline'] = df['Registration Deadline'].str.extract(r'(\s\d+\s\w+,\s\d+)')



In [ ]:
df = df.drop(columns=['Costs'])

In [ ]:
df = df.drop(columns=['Duration'])

In [ ]:
df = df.drop(columns=['Tuition fees per semester'])

In [ ]:
df = df.drop(columns=['C-badge'])

In [1]:
df = df.drop(columns=['Date(s)'])

NameError: ignored

In [ ]:
df["Application deadline"] = df["Application deadline"].fillna("No application deadline")

In [ ]:
df["Financial support"] = df["Financial support"].fillna("No")

In [ ]:
df["Specific support for int. students and doctoral candidates"] = df["Specific support for int. students and doctoral candidates"].fillna("No")

In [ ]:
df["Structured research and supervision"] = df["Structured research and supervision"].fillna("No")

In [ ]:
df['Course enroll End Date'] = pd.to_datetime(df['Course enroll End Date'])
df['Course enroll End Date'] = df['Course enroll End Date'].dt.strftime('%d/%m/%Y')
df['Course enroll Start Date'] = pd.to_datetime(df['Course enroll Start Date'])
df['Course enroll Start Date'] = df['Course enroll Start Date'].dt.strftime('%d/%m/%Y')
df['Registration Deadline'] = pd.to_datetime(df['Registration Deadline'])
df['Registration Deadline'] = df['Registration Deadline'].dt.strftime('%d/%m/%Y')

In [ ]:
df.sample(10).T

,461,1973,361,2019,1461,915,1579,1890,241,905
Course-type,Language course,Preparatory course,Master's degree,Master's degree,Master's degree,Master's degree,Master's degree,Master's degree,Master's degree,Master's degree
Title,Didactics: Teaching German as a Foreign Langu...,Pre-Study@THM,Communications Engineering,Renewable Energy and E-Mobility (REEM),Master of Science in Data Science for Society...,Information Systems - Master of Science,Master of Science in Water Science,North American Studies,Bioeconomy,Informatics: Games Engineering (MSc)
Institution,FH Aachen University of Applied Sciences,TH Mittelhessen University of Applied Science...,University of Duisburg-Essen,Stralsund University of Applied Sciences,Constructor University,University of Cologne,University of Duisburg-Essen,Leibniz Universität Hannover,University of Hohenheim,Technical University Munich
City,Aachen,Gießen,Duisburg,Stralsund,Bremen,Köln,Essen,Hannover,Stuttgart,Garching b. München
Subject,Didactics of German as a Foreign Language,NaN,Electrical Engineering,Electrical Engineering,Computer Science,Computer Science,Chemistry,"English Studies, American Studies","Agriculture, Forestry and Nutritional Sciences...",Computer Science
Language,German,NaN,German,English,English,German,English,English,English,English
Language level,"German: C1, C2",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Beginning,NaN,NaN,"Winter semester, Summer semester","Winter semester, Summer semester",Winter semester,Winter semester,Winter semester,"Winter semester, Summer semester",Winter semester,"Winter semester, Summer semester"
Application deadline,No application deadline,No application deadline,No application deadline,No application deadline,No application deadline,No application deadline,No application deadline,No application deadline,No application deadline,No application deadline
Preparation for,NaN,Bachelor | Master,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
file_path = 'daad_file.csv'
df.to_csv(file_path, index=False)